Import the libraries

In [36]:
import numpy as np
import pandas as pd 

# For portfolio optimization
from scipy.optimize import minimize 
import plotly.graph_objs as go

Reading Finance Data

In [19]:
# import the data from  local file
stock_data=pd.read_excel('Daily_Returns.xlsx')

# Convert the 'date' column to datetime format 
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# Set the 'date' column as the index
stock_data.set_index('Date', inplace=True)

IMP Functions for Portfolio Construction

In [20]:
# get the names of the stocks
def get_stock_names(df):
    stock_names = df.columns[0:].tolist()
    return stock_names

names = get_stock_names(stock_data)

In [21]:
# getting covarince matrix for portfolios

def get_cov_matrix(df):
    # Calculate the covariance matrix for each stock
    cov_matrix = df.cov()
    
    return cov_matrix

# calling the function
cov_matrix = get_cov_matrix(stock_data)

In [22]:
# getting ER of the portfolio

def get_expected_returns(df):
    
    # Calculate the mean return for each stock
    expected_returns = df.mean()
    
    return expected_returns

expected_returns = get_expected_returns(stock_data)

In [23]:
#getting the correlation matrix
def get_cor_matrix(df):
    # Calculate the correlation matrix for each stock
    cor_matrix = df.corr()
    
    return cor_matrix

cor_matrix=get_cor_matrix(stock_data)


Optimally Weighted Portfolios

- To generate optimally weighted portfolis, we use `Mean Variance Optimization`.
- The `SLSQP` algorithm (Sequential Least Squares Programming) allows us to minimize a function with several variables and constraints.
- We are minimizing the negative expected return s.t. 2 constraints are satisfied;
    - Sum of weights should be one
    - Weights should not be negative

In [37]:
number_of_stocks = len(names)
risk_free_rate = ((1+0.03) ** (1/252)) - 1 #conversion into daily RF

# generatig weights
initial_weights = [1/number_of_stocks] * number_of_stocks

constraints = ({'type': 'eq', 'fun': lambda wt: np.sum(wt) - 1}, # sum of weights to be 1
               {'type': 'ineq', 'fun': lambda wt: wt}, # weight should be positive
               {'type': 'ineq', 'fun': lambda wt: 0.1 - np.sqrt(np.dot(wt, np.dot(cov_matrix, wt.T)))}) # risk constraint

bounds = tuple((0, 1) for _ in range(number_of_stocks)) # upper bound is set as 1 for weights
algorithm = 'SLSQP'  #Sequential Least Squares Programming

# defining objective to optimise
def objective(wt, expected_returns):
    return -np.sum(wt * expected_returns)  # maximize return

# generating weights
def get_weights(expected_returns, cov_matrix, initial_weights=initial_weights, 
                algorithm=algorithm,bounds=bounds,constraints=constraints):
    
    result = minimize(objective,
                        initial_weights,
                        args=(expected_returns),
                        method=algorithm,
                        bounds=bounds,
                        constraints=constraints)
    
    return result

In [38]:
# calling the function
weights= get_weights(expected_returns,cov_matrix)

In [57]:
A = np.array(weights.x)
B = (stock_data.cov()).to_numpy()
C = np.sqrt(np.dot(A,np.dot(A,B)))
print(C)

0.018169128041859325


In [39]:
# map weights to stock names

def map_weights_to_names(optimized_weights, names):
    weights_dict = dict(zip(names, optimized_weights))
    
    # Filter out stocks with weights greater than 0
    selected_stocks = {stock: weight for stock, weight in weights_dict.items() if weight > 0}
    
    return selected_stocks

# function call
optimized_weights = weights.x

selected_stocks = map_weights_to_names(optimized_weights, names)

In [42]:
# Sort the dictionary by weights
sorted_stocks = {k: v for k, v in sorted(selected_stocks.items(), key=lambda item: item[1], reverse=True)}

# Print only the top 10 stocks
count = 0
for stock, weight in sorted_stocks.items():
    print(stock, ":", weight)
    count += 1
    if count == 10:
        break

NFLX : 0.30399790168697144
AAPL : 0.11839938876197259
TSLA : 0.11486809977697714
ALGN : 0.09302511362477245
KMX : 0.07793613683898871
BKNG : 0.07668200735026567
DXCM : 0.05961574338755631
ANSS : 0.057181271739711936
DHI : 0.05354129596800163
AMZN : 0.044753040864781574


In [43]:
# Calculate the sum of all weights
total_weight = sum(sorted_stocks.values())

print("Total weight:", total_weight)

Total weight: 0.9999999999999994


In [44]:
# get the list of stocks
stock_names=list(sorted_stocks.keys())
print(len(stock_names))
# get the list of corrosponding weights
stock_weights=list(sorted_stocks.values())
print(len(stock_weights))

# portfolio with selected stocks
df_selected = stock_data[stock_names]

97
97


In [45]:
# Calculate the expected return of the portfolio

ER = get_expected_returns(df_selected)

portfolio_expected_return = np.dot(ER, stock_weights)

# Print the expected return of the portfolio
print("Expected return of the portfolio:", portfolio_expected_return)

Expected return of the portfolio: 0.0012698908074676477


In [32]:
# get a weighted portfolio

# Multiply each stock's return by its corresponding weight
weighted_returns = df_selected.mul(stock_weights)

# Create a new DataFrame with the weighted returns and the 'Date' column
weighted_df = weighted_returns

In [34]:
# calling the function
cov_matrix_weighted = get_cov_matrix(weighted_df)

# get the SD
stock_weights_array = np.array(stock_weights)
portfolio_stddev = np.sqrt(np.dot(stock_weights_array, np.dot(cov_matrix_weighted, stock_weights_array.T)))

# Print the SD of the portfolio
print("SD of the portfolio:", portfolio_stddev)

SD of the portfolio: 0.0035952748433300393
